# Import the necessary libraries

In [ ]:
# Mount your Google Drive if needed

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.9 MB/s eta 0:00:00


In [ ]:
# Note: Some libraries or dependencies used in this code may become deprecated over time.
# If you encounter deprecated libraries, replace them with suitable alternatives and
# update the corresponding sections of the code that rely on the deprecated classes or methods accordingly.

import os
import json
import gzip
import pandas as pd
import collections
import nltk
from google.colab import files

from urllib.request import urlopen

import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
import pickle
from IPython.display import clear_output
import torch
from transformers import RobertaTokenizerFast
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix, matthews_corrcoef
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, AutoTokenizer
# from datasets import load_metric

from typing import Dict, Any, List


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices: {}".format(DEVICE))
print(torch.cuda.get_device_name())


# Load the datasets

In [ ]:
# Ensure you have access to the datasets and update the directory paths as needed.
# Ensure the 'label' AND 'sentence' column exists.
# Rename or update the column if needed.
dataset_1 = pd.read_csv('/content/sample_data/8000_sentences_with_need_IDs.csv')
dataset_1['label'] = dataset_1.pop("isNeed")
dataset_2 = pd.read_csv('/content/sample_data/Electronics.csv')
dataset_3 = pd.read_csv('/content/sample_data/Sport_outdoors.csv')
dataset_4 = pd.read_csv('/content/sample_data/baby.csv')
dataset_5 = pd.read_csv('/content/sample_data/Pet_supplies.csv')

datasets = {"skin-care": dataset_1,
            "Electronics": dataset_2,
            "Sports-outdoors": dataset_3,
            "Baby": dataset_4,
            "pet-supplies" : dataset_5
            }



#  functions needed for training and evaluation

In [ ]:
def tokenize(batch: List[Any]):
    return tokenizer(batch, padding=True, truncation=True, max_length=40)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    probabilities = np.exp(logits) / (1 + np.exp(logits))

    accuracy = accuracy_score(labels, predictions)
    matt = matthews_corrcoef(labels, predictions, sample_weight=None)
    f1 = f1_score(labels, predictions)
    macro_f1 = f1_score(labels, predictions, average= 'macro')
    precision = precision_score(labels, predictions)
    recall = recall_score(labels, predictions)
    auc = roc_auc_score(labels, probabilities[:, 1])

    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()
    specificity = tn / (tn + fp)

    return {
        "accuracy": accuracy,
        "macro_f1": macro_f1,
        "matthews-corr": matt ,
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "specificity": specificity,
        "auc": auc,
    }


def cross_validation_train_and_evaluate(train_df:List[Any], test_dfs:Dict[str, Any], seed, num_folds=5):
    kf = KFold(n_splits = num_folds, shuffle=True, random_state=seed)
    results_all_fold = {}
    for n_fold, (train_index, val_index) in enumerate(kf.split(train_df[1])):
        results = {}
        if train_df[0] != "skin-care":
           train_data = train_df[1].iloc[train_index]
           val_data = train_df[1].iloc[val_index]
        else:
           train_data = train_df[1].iloc[val_index]
           val_data = train_df[1].iloc[train_index]

        train_encodings = tokenize(train_data['sentence'].tolist())
        val_encodings = tokenize(val_data['sentence'].tolist())

        train_dataset = TrainerDataset(train_encodings, train_data['label'].tolist())
        val_dataset = TrainerDataset(val_encodings, val_data['label'].tolist())

        # The best setting from Table_3 results has been chosen for training.
        model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=2)
        training_args = TrainingArguments(
            output_dir=f"./results_{train_df[0]}-{seed}",
            logging_strategy="steps",
            logging_steps=50,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=128,
            weight_decay=0.01,
            num_train_epochs=7,
            seed=seed,
            learning_rate = 1e-5,
            load_best_model_at_end = False,
            save_strategy = "no",
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            # eval_dataset=val_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
        )

        trainer.train()
        results[train_df[0]] = trainer.evaluate(val_dataset)
        for name_of_dataset, test_df in test_dfs.items():
            test_encodings = tokenize(test_df['sentence'].tolist())
            test_dataset = TrainerDataset(test_encodings, test_df['label'].tolist())
            results[name_of_dataset]  = trainer.evaluate(test_dataset)
        results_all_fold[n_fold + 1] = results

    return results_all_fold


class TrainerDataset:
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


def set_seed(SEED):
    """ Set random seed to all """
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True

# Run ( cross-domain evaluation)

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-large")

CD_all_seed_fold_results = []
num_seeds = [94, 791, 5, 6932, 1759, 323, 1694, 9741, 200, 999]
num_folds = 5

metrics = ["eval_accuracy", "eval_macro_f1", "eval_matthews-corr", "eval_f1", "eval_precision", "eval_recall", "eval_specificity", "eval_auc"]

for k, train_df in datasets.items():
    print(f"training for dataset {k}")
    test_dfs = {d_name: df for d_name, df in datasets.items() if d_name != k}  # Exclude the train_df from test_dfs
    for seed in tqdm(num_seeds):
        set_seed(seed)
        res = cross_validation_train_and_evaluate([k , train_df], test_dfs, seed, num_folds)
        for f_id, fold_res in res.items():
            for j, test_result in fold_res.items():
               result_dict = {"seed": seed, "train_dataset": k, "fold": f_id, "test_dataset": j}
               for metric in metrics:
                   result_dict[metric] = test_result[metric]
               CD_all_seed_fold_results.append(result_dict)


results_df = pd.DataFrame(CD_all_seed_fold_results)
results_df.to_csv("all_cross_domain_evaluation_results.csv", index=False)

# Calculate the average results
average_results_df = results_df.groupby(["train_dataset", "test_dataset"]).mean()
average_results_df.to_csv("average_cross_domain_evaluation_results.csv")


training for dataset skin-care


<ipython-input-7-03924f538f59>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for seed in tqdm(num_seeds):


  0%|          | 0/10 [00:00<?, ?it/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

Step,Training Loss
50,0.689800
100,0.615700
150,0.494600
200,0.501300
250,0.346600
300,0.362700
350,0.222000
400,0.228800
450,0.151200
500,0.155700


...

Step,Training Loss
50,0.625000
100,0.417200
150,0.355800
200,0.290400
250,0.290300
300,0.237700
350,0.170200
400,0.216700
450,0.114800
500,0.109900


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

Step,Training Loss
50,0.642500
100,0.423100
150,0.358900
200,0.300700
250,0.239900
300,0.282000
350,0.183000
400,0.197900
450,0.123300
500,0.153700


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

Step,Training Loss
50,0.572300
100,0.396700
150,0.328400
200,0.319400
250,0.209400
300,0.266200
350,0.133900
400,0.134500
450,0.070500
500,0.100200


In [ ]:
# Output sample result
average_results_df


seed  fold  eval_accuracy  eval_macro_f1  \
train_dataset   test_dataset                                                  
Baby            Baby             2253.8   3.0       0.816700       0.814853   
                Electronics      2253.8   3.0       0.767630       0.746365   
                Sports-outdoors  2253.8   3.0       0.765150       0.753091   
                pet-supplies     2253.8   3.0       0.775050       0.746750   
                skin-care        2253.8   3.0       0.653457       0.650983   
Electronics     Baby             2253.8   3.0       0.798060       0.793185   
                Electronics      2253.8   3.0       0.806300       0.800518   
                Sports-outdoors  2253.8   3.0       0.780710       0.759719   
                pet-supplies     2253.8   3.0       0.823780       0.799825   
                skin-care        2253.8   3.0       0.690050       0.688869   
Sports-outdoors Baby             2253.8   3.0       0.784730       0.778141   
                Electronics      2253.8   3.0       0.746050       0.716419   
                Sports-outdoors  2253.8   3.0       0.802450       0.787895   
                pet-supplies     2253.8   3.0       0.805640       0.783353   
                skin-care        2253.8   3.0       0.679310       0.675513   
pet-supplies    Baby             2253.8   3.0       0.773760       0.763474   
                Electronics      2253.8   3.0       0.689470       0.588782   
                Sports-outdoors  2253.8   3.0       0.785040       0.757568   
                pet-supplies     2253.8   3.0       0.859850       0.844009   
                skin-care        2253.8   3.0       0.669680       0.662561   
skin-care       Baby             2253.8   3.0       0.741590       0.741132   
                Electronics      2253.8   3.0       0.765860       0.756841   
                Sports-outdoors  2253.8   3.0       0.766450       0.756162   
                pet-supplies     2253.8   3.0       0.779520       0.769739   
                skin-care        2253.8   3.0       0.775022       0.772831   

                                 eval_matthews-corr   eval_f1  eval_precision  \
train_dataset   test_dataset                                                    
Baby            Baby                       0.635603  0.797634        0.756748   
                Electronics                0.504825  0.673894        0.749948   
                Sports-outdoors            0.510243  0.698900        0.663536   
                pet-supplies               0.498264  0.662755        0.636200   
                skin-care                  0.321891  0.626614        0.715924   
Electronics     Baby                       0.587646  0.761922        0.769984   
                Electronics                0.605532  0.767300        0.726745   
                Sports-outdoors            0.522312  0.688924        0.723289   
                pet-supplies               0.603269  0.731029        0.714097   
                skin-care                  0.391550  0.674336        0.747622   
Sports-outdoors Baby                       0.565279  0.749036        0.718202   
                Electronics                0.453006  0.627448        0.715710   
                Sports-outdoors            0.585726  0.736797        0.697861   
                pet-supplies               0.582674  0.717026        0.649740   
                skin-care                  0.367917  0.658392        0.718066   
pet-supplies    Baby                       0.534491  0.714547        0.776336   
                Electronics                0.340433  0.385726        0.853325   
                Sports-outdoors            0.526269  0.676163        0.761118   
                pet-supplies               0.693320  0.794580        0.747160   
                skin-care                  0.376060  0.614357        0.787497   
skin-care       Baby                       0.509248  0.737594        0.654322   
                Electronics           

In [ ]:
results_df[(results_df.train_dataset == 'Electronics') & (results_df.test_dataset == 'Electronics')]


,seed,train_dataset,fold,test_dataset,eval_accuracy,eval_macro_f1,eval_matthews-corr,eval_f1,eval_precision,eval_recall,eval_specificity,eval_auc
250,94,Electronics,1,Electronics,0.8075,0.800868,0.604812,0.764526,0.726744,0.806452,0.808163,0.891007
255,94,Electronics,2,Electronics,0.8050,0.801102,0.612893,0.773256,0.707447,0.852564,0.774590,0.893416
260,94,Electronics,3,Electronics,0.8125,0.807731,0.625671,0.777448,0.711957,0.856209,0.785425,0.876796
265,94,Electronics,4,Electronics,0.8500,0.845261,0.690701,0.818182,0.808383,0.828221,0.864979,0.913127
270,94,Electronics,5,Electronics,0.7725,0.767785,0.541807,0.734694,0.684783,0.792453,0.759336,0.866437
275,791,Electronics,1,Electronics,0.8275,0.821215,0.643754,0.787692,0.761905,0.815287,0.835391,0.896543
280,791,Electronics,2,Electronics,0.8100,0.799388,0.598819,0.753247,0.758170,0.748387,0.848980,0.883529
285,791,Electronics,3,Electronics,0.8275,0.820498,0.644982,0.785047,0.741176,0.834437,0.823293,0.901061
290,791,Electronics,4,Electronics,0.8425,0.839897,0.683708,0.819484,0.777174,0.866667,0.825532,0.917086
295,791,Electronics,5,Electronics,0.7775,0.772202,0.549739,0.737463,0.690608,0.791139,0.768595,0.858824


In [ ]:
files.download("/content/average_cross_domain_evaluation_results.csv")
files.download("/content/all_cross_domain_evaluation_results.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>